# Accelerated Parameter Mapping of Multiple-Echo Gradient-Echo Data Using Model-Based Iterative Reconstruction

*M. Zimmermann, Z. Abbas, K. Dzieciol and N. J. Shah, "Accelerated Parameter Mapping of Multiple-Echo Gradient-Echo Data Using Model-Based Iterative Reconstruction," in IEEE Transactions on Medical Imaging, vol. 37, no. 2, pp. 626-637, Feb. 2018, doi: 10.1109/TMI.2017.2771504.*

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os
import sys

sys.path.insert(0, "../src")

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import h5py
import ismrmrd
import matplotlib.pyplot as plt
import torch

from juart.conopt.functional.fourier import fourier_transform_adjoint
from juart.preproc.aux import process_siemens_folder

# from juart.preproc.aux import process_siemens_file
from juart.preproc.data import KSpaceData, get_shape
from juart.preproc.trajectory import KSpaceTrajectory

torch.set_num_threads(1)

In [2]:
# set folder to convert to h5d

# fname = "7T1026"
foldername = '/workspaces/juart/LocalData/'

In [3]:
# save = True
# pulseq = False

In [4]:
process_siemens_folder(foldername)

Skipping file /workspaces/juart/LocalData/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5 as it already exists.


0

In [5]:
full_session_dir = foldername
h5_ismrmrd_fname = "meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5"
h5_preproc_fname = "preproc/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5"

In [6]:
full_h5_ismrmrd_fname = os.path.join(full_session_dir, h5_ismrmrd_fname)
full_h5_preproc_fname = os.path.join(full_session_dir, h5_preproc_fname)

In [7]:
print(full_h5_ismrmrd_fname)
print(full_h5_preproc_fname)

/workspaces/juart/LocalData/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5
/workspaces/juart/LocalData/preproc/meas_MID00101_FID125842_CS_MP2RAGE_0_6mm_proposedTiming.h5


# Load Data

In [8]:
print("(1/2) Loading rawdata ...")

(1/2) Loading rawdata ...


In [9]:
dataset = ismrmrd.Dataset(
    full_h5_ismrmrd_fname,
    dataset_name="dataset",
    create_if_needed=False,
)

In [24]:
NCha, NCol, NLin, NPar, NSli, NSet, NEco = get_shape(dataset)

In [11]:
# For now, manually set this to 160
# NPar = 160

In [25]:
print(NCha, NCol, NLin, NPar, NSli, NSet, NEco)

32 128 360 288 1 2 1


In [ ]:
# NLin_retro = 8
# NPar_post, NSli_post = NSli, NPar
# NCha_comp, ISet_comp, IEco_comp = 8, slice(NSet - 1, NSet), slice(0, 1)
# NImx, NImy, ISet_coil, IEco_coil = 256, 256, slice(15, 19), slice(0, 1)
# NAcl, NUsf = 32, 2
# NCol = 608

In [33]:
max_kspace_encode_step_1 = 0
max_kspace_encode_step_2 = 0
max_average = 0
max_slice = 0
max_contrast = 0
max_phase = 0
max_repetition = 0
max_set = 0
max_segment = 0


for index in range(dataset.number_of_acquisitions()):
    # Read the acquisition.
    acquisition = dataset.read_acquisition(index)
    if( acquisition.idx.kspace_encode_step_1 > max_kspace_encode_step_1):
        max_kspace_encode_step_1 = acquisition.idx.kspace_encode_step_1
    if( acquisition.idx.kspace_encode_step_2 > max_kspace_encode_step_2):
        max_kspace_encode_step_2 = acquisition.idx.kspace_encode_step_2
    if( acquisition.idx.average > max_average):
        max_average = acquisition.idx.average
    if( acquisition.idx.slice > max_slice):
        max_slice = acquisition.idx.slice
    if( acquisition.idx.contrast > max_contrast):
        max_contrast = acquisition.idx.contrast
    if( acquisition.idx.phase > max_phase):
        max_phase = acquisition.idx.phase
    if( acquisition.idx.repetition > max_repetition):
        max_repetition = acquisition.idx.repetition
    if( acquisition.idx.set > max_set):
        max_set = acquisition.idx.set
    if( acquisition.idx.segment > max_segment):
        max_segment = acquisition.idx.segment


In [42]:
print(max_kspace_encode_step_1)     # phase in plane
print(max_kspace_encode_step_2)     # slice
# print(max_average)
# print(max_slice)
# print(max_contrast)
# print(max_phase)
# print(max_repetition)
# print(max_set)
# print(max_segment)

359
287


In [ ]:
# Images 360 x 384 x 288 (Phase x Frequency x Slices) 

In [43]:
acquisition.data.shape[1]

608

In [26]:
acquisition.data.shape

(32, 608)

In [ ]:
# kdata = torch.zeros((NCha, NCol, NLin, NPar, NSli, NSet, NEco), dtype=torch.complex64)

In [ ]:
for index in range(dataset.number_of_acquisitions()):

    # Read the acquisition.
    acquisition = dataset.read_acquisition(index)

    
    if acquisition.data.shape == kdata.shape[:2]:
        # Update shared memory.
        kdata[
            :,
            :,
            acquisition.idx.kspace_encode_step_1,
            acquisition.idx.kspace_encode_step_2,
            acquisition.idx.slice,
            acquisition.idx.set,
            acquisition.idx.contrast,
        ] = torch.tensor(acquisition.data)
    else:
        print(index, acquisition.data.shape)

In [ ]:
kdata = KSpaceData((NCha, NCol, NLin, NPar, NSli, NSet, NEco))

In [ ]:
kdata.read_data(dataset, is_pulseq=False)

In [ ]:
# Undersample along line dimension
kdata.index_data([slice(None), slice(None), slice(NLin_retro), ...])

In [ ]:
plt.figure()
plt.imshow(torch.log(torch.abs(kdata.kdata[0, :, 0, :, 0, 0, 0]) + 1e-16).numpy())

# GRAPPA

In [ ]:
kdata.apply_mask(NUsf, NAcl, is_pulseq=True)

In [ ]:
plt.figure()
plt.imshow(torch.log(torch.abs(kdata.kdata[0, :, 0, :, 0, 0, 0]) + 1e-16).numpy())

In [ ]:
kdata.reconstruct_partitions()

In [ ]:
plt.figure()
plt.imshow(torch.log(torch.abs(kdata.kdata[0, :, 0, :, 0, 0, 0]) + 1e-16).numpy())

# Coil compression

In [ ]:
# Swap Par and Sli Partition
kdata.swapaxes(3, 4)

In [ ]:
comp_matrix = kdata.compression_matrix(NCha_comp, ISet_comp, IEco_comp)

In [ ]:
kdata.compress_data(comp_matrix)

In [ ]:
# Swap Col and Lin dimension
kdata.swapaxes(1, 2)

In [ ]:
NCha, NLin, NCol, NPar, NSli, NSet, NEco = kdata.kdata_shape

In [ ]:
print("(4/4) Creating output file ....")

if save:
    with h5py.File(full_h5_preproc_fname, "w", libver="latest") as out_file:
        out_file.create_dataset(
            "d", data=kdata.kdata.numpy(), chunks=(NCha, NLin, NCol, 1, 1, 1, 1)
        )

In [ ]:
plt.figure()
plt.subplot(3, 2, 1)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[0, 0, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 2)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[-1, 0, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 3)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[0, 1, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 4)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[-1, 1, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 5)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[0, 2, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")
plt.subplot(3, 2, 6)
plt.imshow(
    torch.log(torch.abs(kdata.kdata[-1, 2, :, 0, :, 0, 0].T)).numpy(), vmin=-16, vmax=-7
)
plt.axis("off")

In [ ]:
vmax = 5e-4

In [ ]:
plt.figure()
plt.subplot(3, 2, 1)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[0, 0, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 2)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[-1, 0, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 3)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[0, 1, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 4)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[-1, 1, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 5)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[0, 2, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")
plt.subplot(3, 2, 6)
plt.imshow(
    torch.abs(
        fourier_transform_adjoint(kdata.kdata[-1, 2, :, 0, :, 0, 0], axes=(0,)).T
    ).numpy(),
    vmin=0,
    vmax=vmax,
)
plt.axis("off")

# Gradient Delay Correction (RING)

In [ ]:
ktraj = KSpaceTrajectory(NLin, NCol, NSet, NEco)

In [ ]:
S_corr = ktraj.estimate_gradient_delay(kdata.kdata)

In [ ]:
S_corr

In [ ]:
ktraj_corr = ktraj.correct_kspace_trajectory(S_corr)

In [ ]:
if save:
    with h5py.File(full_h5_preproc_fname, "r+", libver="latest") as h5_preproc_file:
        dset = h5_preproc_file.create_dataset(
            "k", data=ktraj_corr.numpy(), chunks=(2, NLin_retro, NCol, 1, 1, 1, 1)
        )

# Coil Sensitivity Estimation (SAKE + ESPIRiT)

In [ ]:
sensmaps = kdata.get_sensmaps(ktraj_corr, (NCha, NImx, NImy, 1, NSli, 1, 1))

In [ ]:
if save:
    with h5py.File(full_h5_preproc_fname, "r+", libver="latest") as h5_preproc_file:
        dset = h5_preproc_file.create_dataset(
            "C",
            data=sensmaps.numpy(),
            chunks=(NCha_comp, NImx, NImy, NPar_post, NSli_post, 1, 1),
        )

In [ ]:
plt.figure()
plt.subplot(1, 3, 1)
plt.imshow(torch.abs(sensmaps[0, :, :, 0, 80, 0, 0]).numpy())
plt.subplot(1, 3, 2)
plt.imshow(torch.abs(sensmaps[0, :, 128, 0, :, 0, 0]).T.numpy())
plt.subplot(1, 3, 3)
plt.imshow(torch.abs(sensmaps[0, 128, :, 0, :, 0, 0]).T.numpy())

In [ ]:
plt.figure()
plt.subplot(1, 2, 1)
plt.imshow(torch.abs(sensmaps[0, :, 128, 0, :, 0, 0].T).numpy(), vmin=0, vmax=1)
plt.axis("off")
plt.subplot(1, 2, 2)
plt.imshow(torch.abs(sensmaps[-1, :, 128, 0, :, 0, 0].T).numpy(), vmin=0, vmax=1)
plt.axis("off")